# Descripción del proyecto

Encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Mediante los siguientes pasos:

. Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.

. Crear un modelo para predecir el volumen de reservas en pozos nuevos.

. Elegir los pozos petrolíferos que tienen los valores estimados más altos.

. Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.

. Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. 

. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

In [44]:
# Importar librerias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [45]:
# Cargar los datos
file_paths = {
    'region 0': '/datasets/geo_data_0.csv',
    'region 1': '/datasets/geo_data_1.csv',
    'region 2': '/datasets/geo_data_2.csv'
}

In [46]:
# Leer los archivos y mostrar información básica
data_info = {}
for region, path in file_paths.items():
    df = pd.read_csv(path)
    data_info[region] = {
        "shape": df.shape,
        "columns": df.columns.tolist(),
        "missing_values": df.isnull().sum().to_dict(),
        "preview": df.head()
    }

data_info

{'region 0': {'shape': (100000, 5),
  'columns': ['id', 'f0', 'f1', 'f2', 'product'],
  'missing_values': {'id': 0, 'f0': 0, 'f1': 0, 'f2': 0, 'product': 0},
  'preview':       id        f0        f1        f2     product
  0  txEyH  0.705745 -0.497823  1.221170  105.280062
  1  2acmU  1.334711 -0.340164  4.365080   73.037750
  2  409Wp  1.022732  0.151990  1.419926   85.265647
  3  iJLyR -0.032172  0.139033  2.978566  168.620776
  4  Xdl7t  1.988431  0.155413  4.751769  154.036647},
 'region 1': {'shape': (100000, 5),
  'columns': ['id', 'f0', 'f1', 'f2', 'product'],
  'missing_values': {'id': 0, 'f0': 0, 'f1': 0, 'f2': 0, 'product': 0},
  'preview':       id         f0         f1        f2     product
  0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
  1  62mP7  14.272088  -3.475083  0.999183   26.953261
  2  vyE1P   6.263187  -5.948386  5.001160  134.766305
  3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
  4  AHL4O  12.702195  -8.147433  5.004363  134.766305},
 'region 2

In [47]:
# Entrenar y evaluar el modelo en una región
def train_and_evaluate(region_name, df):
    # Separar características y objetivo
    X = df[['f0', 'f1', 'f2']]
    y = df['product']
    
    # Dividir en conjunto de entrenamiento y validación (75:25)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)
    
    # Entrenar modelo de regresión lineal
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Hacer predicciones
    predictions = model.predict(X_valid)
    
    # Calcular RMSE
    rmse = mean_squared_error(y_valid, predictions, squared=False)
    
    return {
        "region": region_name,
        "model": model,
        "rmse": rmse,
        "mean_prediction": predictions.mean(),
        "predictions": predictions,
        "actuals": y_valid.values
    }

# Aplicar la función a cada región.
results = {}
for region, path in file_paths.items():
    df = pd.read_csv(path)
    results[region] = train_and_evaluate(region, df)

results

{'region 0': {'region': 'region 0',
  'model': LinearRegression(),
  'rmse': 37.756600350261685,
  'mean_prediction': 92.3987999065777,
  'predictions': array([101.90101715,  78.21777385, 115.26690103, ...,  82.54439653,
          81.82668931,  93.12106221]),
  'actuals': array([122.07334983,  48.73853962, 131.33808824, ...,  91.94521309,
         149.29556326,  57.24403851])},
 'region 1': {'region': 'region 1',
  'model': LinearRegression(),
  'rmse': 0.890280100102884,
  'mean_prediction': 68.71287803913762,
  'predictions': array([  0.84473806,  52.92161194, 135.11038454, ...,  26.70873415,
         109.82308735, 135.44878039]),
  'actuals': array([  0.        ,  53.90652206, 134.76630516, ...,  26.95326103,
         110.99214671, 134.76630516])},
 'region 2': {'region': 'region 2',
  'model': LinearRegression(),
  'rmse': 40.14587231134218,
  'mean_prediction': 94.77102387765939,
  'predictions': array([ 98.30191642, 101.59246124,  52.4490989 , ...,  64.09839   ,
          83.7641

**Resultados del entrenamiento y evaluación por región:**

* Región 0:

RMSE: 37.76

Volumen medio de reservas predicho: 92.40

* Región 1:

RMSE: 0.89 (el más bajo, indicando mejor precisión)

Volumen medio de reservas predicho: 68.71

* Región 2:

RMSE: 40.15

Volumen medio de reservas predicho: 94.77


* Observación:
La Región 1 tiene el modelo más preciso (RMSE más bajo), es decir las predicciones son más confiables (se ajusta mejor a los datos)

# Cálculo de ganancias

Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades).

Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).

Cada pozo debe producir al menos 111.1 unidades de reservas para generar 500,000 dólares y evitar pérdidas.

In [48]:
# Parámetros para el cálculo del beneficio
cost_per_well = 500000  # Costo por pozo en dólares
revenue_per_unit = 4500  # Ingreso por unidad de volumen de reservas en dólares
threshold_units = 111.1  # Umbral mínimo de unidades para evitar pérdidas

# Volumen medio de reservas por región (calculado previamente)
mean_reserves_0 = results['region 0']['mean_prediction']
mean_reserves_1 = results['region 1']['mean_prediction']
mean_reserves_2 = results['region 2']['mean_prediction']

In [49]:
# Comparar el umbral mínimo con el volumen medio de reservas
print('Comparación del umbral mínimo (111.1 unidades) con el volumen medio de reservas:')
print(f'Región 0 - Volumen medio: {mean_reserves_0}, Cumple con el umbral: {mean_reserves_0 >= threshold_units}')
print(f'Región 1 - Volumen medio: {mean_reserves_1}, Cumple con el umbral: {mean_reserves_1 >= threshold_units}')
print(f'Región 2 - Volumen medio: {mean_reserves_2}, Cumple con el umbral: {mean_reserves_2 >= threshold_units}')

Comparación del umbral mínimo (111.1 unidades) con el volumen medio de reservas:
Región 0 - Volumen medio: 92.3987999065777, Cumple con el umbral: False
Región 1 - Volumen medio: 68.71287803913762, Cumple con el umbral: False
Región 2 - Volumen medio: 94.77102387765939, Cumple con el umbral: False


Dicho de otra manera:

Región 0:

Volumen medio de reservas: 92.40 unidades.

No cumple con el umbral mínimo de 111.1 unidades.



Región 1:

Volumen medio de reservas: 68.71 unidades.

No cumple con el umbral mínimo de 111.1 unidades.



Región 2:

Volumen medio de reservas: 94.77 unidades.

No cumple con el umbral mínimo de 111.1 unidades.


* Observación:
Ninguna región cumple con el umbral mínimo de 111.1 unidades de volumen de reservas. 

# Seleccionar los 500 Puntos y Elegir los Mejores 200 para el cálculo del beneficio

In [50]:
# Fijar la semilla para reproducibilidad
np.random.seed(12345)

def select_top_wells(predictions, actuals, n=500, top_n=200):
    # Seleccionar 500 pozos aleatorios (con semilla fija)
    random_indices = np.random.choice(len(predictions), size=n, replace=False)
    random_predictions = predictions[random_indices]
    random_actuals = actuals[random_indices]
    
    # Seleccionar los 200 mejores de los 500
    top_indices = np.argsort(random_predictions)[-top_n:]
    top_actuals = random_actuals[top_indices]
    return top_actuals

# Seleccionar los 200 mejores pozos de 500 para cada región
top_wells_0 = select_top_wells(results['region 0']['predictions'], results['region 0']['actuals'])
top_wells_1 = select_top_wells(results['region 1']['predictions'], results['region 1']['actuals'])
top_wells_2 = select_top_wells(results['region 2']['predictions'], results['region 2']['actuals'])

Establecí una semilla, para eliminar la variabilidad de los resultados y puedan ser reproducibles.

# Calcular el Beneficio Potencial

In [51]:
# Parámetros para el cálculo del beneficio
cost_per_well = 500000  # Costo por pozo en dólares
revenue_per_unit = 4500  # Ingreso por unidad de volumen de reservas en dólares

def calculate_profit(top_wells, cost_per_well, revenue_per_unit):
    total_volume = top_wells.sum()
    total_cost = cost_per_well * len(top_wells)
    total_revenue = total_volume * revenue_per_unit
    profit = total_revenue - total_cost
    return profit

# Calcular el beneficio para cada región
profit_0 = calculate_profit(top_wells_0, cost_per_well, revenue_per_unit)
profit_1 = calculate_profit(top_wells_1, cost_per_well, revenue_per_unit)
profit_2 = calculate_profit(top_wells_2, cost_per_well, revenue_per_unit)

print('Beneficio potencial para los 200 pozos seleccionados:')
print(f'Región 0: {profit_0:,.2f} USD')
print(f'Región 1: {profit_1:,.2f} USD')
print(f'Región 2: {profit_2:,.2f} USD')

Beneficio potencial para los 200 pozos seleccionados:
Región 0: 3,748,191.66 USD
Región 1: 5,105,278.06 USD
Región 2: 7,590,703.77 USD


* Observación:
La Región 2 tiene el beneficio potencial más alto (7,590,703.77 USD), seguida de la Región 1 y la Región 0.


* NOTA: En los pasos anteriores la Region 1 estaría siendo la mejor region para abrir los 200 pozos, sin embargo, al realizar el análisis del beneficio potencial nos damos cuenta que ahora la Region 2 tiene mejor beneficio.

# Análisis de Riesgos con Bootstrapping.

Utilizaremos bootstrapping para estimar la distribución de los beneficios y calcular el riesgo de pérdidas. 
Solo consideraremos regiones con un riesgo de pérdidas inferior al 2.5%.

In [52]:
def bootstrap_profit(top_wells, cost_per_well, revenue_per_unit, n_samples=1000):
    profits = []
    for i in range(n_samples):
        # Muestrear con reemplazo
        sample = np.random.choice(top_wells, size=len(top_wells), replace=True)
        # Calcular el beneficio para la muestra
        profit = calculate_profit(sample, cost_per_well, revenue_per_unit)
        profits.append(profit)
    return np.array(profits)

# Aplicar bootstrapping a cada región
profits_0 = bootstrap_profit(top_wells_0, cost_per_well, revenue_per_unit)
profits_1 = bootstrap_profit(top_wells_1, cost_per_well, revenue_per_unit)
profits_2 = bootstrap_profit(top_wells_2, cost_per_well, revenue_per_unit)

# Calcular el beneficio promedio, intervalo de confianza del 95% y riesgo de pérdidas
def analyze_profits(profits):
    mean_profit = profits.mean()
    confidence_interval = np.percentile(profits, [2.5, 97.5])
    risk_of_loss = (profits < 0).mean() * 100  # Probabilidad de pérdida en porcentaje
    return mean_profit, confidence_interval, risk_of_loss

# Analizar los beneficios para cada región
analysis_0 = analyze_profits(profits_0)
analysis_1 = analyze_profits(profits_1)
analysis_2 = analyze_profits(profits_2)

print('Análisis de beneficios y riesgos:')
print(f'Región 0 - Beneficio promedio: {analysis_0[0]}, Intervalo de confianza: {analysis_0[1]}, Riesgo de pérdidas: {analysis_0[2]}%')
print(f'Región 1 - Beneficio promedio: {analysis_1[0]}, Intervalo de confianza: {analysis_1[1]}, Riesgo de pérdidas: {analysis_1[2]}%')
print(f'Región 2 - Beneficio promedio: {analysis_2[0]}, Intervalo de confianza: {analysis_2[1]}, Riesgo de pérdidas: {analysis_2[2]}%')

Análisis de beneficios y riesgos:
Región 0 - Beneficio promedio: 3961956.3517637276, Intervalo de confianza: [-984870.52469525 9005605.3213894 ], Riesgo de pérdidas: 5.4%
Región 1 - Beneficio promedio: 5055775.697164308, Intervalo de confianza: [2714300.92014181 7603845.35399855], Riesgo de pérdidas: 0.0%
Región 2 - Beneficio promedio: 7657068.948077464, Intervalo de confianza: [ 2534942.19547236 12349928.8666193 ], Riesgo de pérdidas: 0.0%


**Análisis de Riesgos**
* Región 0:

Beneficio promedio: 3,961,956.35 USD

Intervalo de confianza: [-984,870.52, 9,005,605.32]

Riesgo de pérdidas: 5.4%

El riesgo de pérdidas es superior al 2.5%, lo que no cumple con los criterios establecidos.


* Región 1:

Beneficio promedio: 5,055,775.70 USD

Intervalo de confianza: [2,714,300.92, 7,603,845.35]

Riesgo de pérdidas: 0.0%

El riesgo de pérdidas es 0%, lo que cumple con los criterios. Además, tiene un beneficio promedio alto.


* Región 2:

Beneficio promedio: 7,657,068.95 USD

Intervalo de confianza: [2,534,942.20, 12,349,928.87]

Riesgo de pérdidas: 0.0%




* NOTA: El intervalo de confiaza en la Region 0 tiene un valor negativo, esto puede deberse a que los pozos seleccionados en la muestra tienen un volumen de reservas muy bajo y/o los costos superan los ingresos en esa muestra.


# Selección de la Región
Filtraremos las regiones que cumplen con el criterio de riesgo de pérdidas inferior al 2.5% y seleccionaremos la que tenga el beneficio promedio más alto.

In [53]:
# Filtrar regiones con riesgo de pérdidas < 2.5%
filtered_regions = []
for region, analysis in zip(['region 0', 'region 1', 'region 2'], [analysis_0, analysis_1, analysis_2]):
    if analysis[2] < 2.5:
        filtered_regions.append((region, analysis[0]))

# Seleccionar la región con el beneficio promedio más alto
if filtered_regions:
    best_region = max(filtered_regions, key=lambda x: x[1])
    print(f'La mejor región es {best_region[0]} con un beneficio promedio de {best_region[1]:,.2f} USD')
else:
    print('Ninguna región cumple con el criterio de riesgo de pérdidas < 2.5%.')

La mejor región es region 2 con un beneficio promedio de 7,657,068.95 USD


# Conclusión

De acuerdo al objetivo planteado por la compañia de extracción de petróleo OilyGiant en este proyecto, el cual es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo, el resultado concluyente es que la Region 2 es la mejor opción ya que tiene el beneficio promedio más alto (7,657,068.95 USD). Para llegar a esta conclusión evaluamos el riesgo de pérdidas = 0%, lo que cumple con el criterio de ser inferior al 2.5%. El beneficio potencial y el riesgo son más favorables en la Region 2 incluso aunque su RMSE es más alto que en la Region 1. Descartamos la Region 0 ya que no cumple con los criterios debido a su alto riesgo de pérdidas (5.4%).

Por lo tanto, seleccionar la Region 2 espara el desarrollo de los 200 pozos petroleros es lo recomendable, siendo la opción más rentable y segura.